In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp

DATA_DIR = os.path.join(os.path.abspath(""), "data", "2023")
GENDER_PREFIX = "M" #"W"  # data pertaining to men is prepended by 'M', women by 'W'

teams_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}Teams.csv"))
reg_season_compact_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}RegularSeasonCompactResults.csv"))
reg_season_detailed_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}RegularSeasonDetailedResults.csv"))
sample_submission_df = pd.read_csv(os.path.join(DATA_DIR, "SampleSubmission2023.csv"))
seasons_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}Seasons.csv"))
tourney_compact_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}NCAATourneyCompactResults.csv"))
tourney_detailed_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}NCAATourneyDetailedResults.csv"))
seeds_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}NCAATourneySeeds.csv"))
slots_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}NCAATourneySlots.csv"))
coaches_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}TeamCoaches.csv"))
conferences_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}TeamConferences.csv"))
massey_ordinals_df = pd.read_csv(os.path.join(DATA_DIR, f"{GENDER_PREFIX}MasseyOrdinals.csv"))

df_list = [
    teams_df, reg_season_compact_df, reg_season_detailed_df, sample_submission_df, seasons_df,
    tourney_compact_df, tourney_detailed_df, seeds_df, slots_df, coaches_df,
    conferences_df, massey_ordinals_df
]


In [2]:
team = teams_df[teams_df["TeamName"] == "Penn St"]
team_id = team["TeamID"].values[0]
detailed_seasons = [yr for yr in range(2003, 2024)]
year = 2003

year_df = reg_season_detailed_df[reg_season_detailed_df["Season"] == year]
massey_year_df = massey_ordinals_df[massey_ordinals_df["Season"] == year]

team_year_df = year_df[(year_df['LTeamID'] == team_id) | (year_df["WTeamID"] == team_id)]
conference = conferences_df[(conferences_df["Season"] == year) & (conferences_df["TeamID"] == team_id)]["ConfAbbrev"].values[0]

team_year_df["opp_team_id"] = np.where(team_year_df['LTeamID'] == team_id, team_year_df["WTeamID"], team_year_df["LTeamID"])

team_year_w_massey = []
for idx, row in team_year_df.iterrows():
    opp_team_id = row["opp_team_id"]
    row["opp_massey_avg"] = massey_year_df[massey_year_df["TeamID"] == opp_team_id]["OrdinalRank"].mean()
    team_year_w_massey.append(row)

team_year_df_full = pd.DataFrame(team_year_w_massey)

/var/folders/96/v69ct1597978t9sjzmlk4cb00000gn/T/ipykernel_1314/2838259518.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_year_df["opp_team_id"] = np.where(team_year_df['LTeamID'] == team_id, team_year_df["WTeamID"], team_year_df["LTeamID"])


In [6]:
aggregated_cols = ["Score", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]

for stat in aggregated_cols:

    stat_yearly_avg_per_game = f"{stat.lower()}_pg"
    stat_yearly_standard_error = f"{stat.lower()}_se"
    stat_yearly_skewness = f"{stat.lower()}_skew"
    stat_yearly_avg_per_game_opp = f"opp_{stat.lower()}_pg"
    stat_yearly_standard_error_opp = f"{stat.lower()}_se"
    stat_yearly_skewness_opp = f"{stat.lower()}_skew"

    win_stat_name = f"W{stat}"
    lose_stat_name = f"L{stat}"

    stat_list = list(team_year_df_full[team_year_df_full["WTeamID"] == team_id][win_stat_name]) + list(team_year_df_full[team_year_df_full["LTeamID"] == team_id][lose_stat_name])
    stat_per_game = np.mean(stat_list)
    stat_std = np.std(stat_list)
    stat_skewness = sp.stats.skew(stat_list)

    opp_stat_list = list(team_year_df_full[team_year_df_full["WTeamID"] == team_id][lose_stat_name]) + list(team_year_df_full[team_year_df_full["LTeamID"] == team_id][win_stat_name])
    opp_stat_per_game = np.mean(opp_stat_list)
    opp_stat_std = np.std(opp_stat_list)
    opp_stat_skewness = sp.stats.skew(opp_stat_list)


In [9]:
win_stat_name = "WScore"
lose_stat_name = "LScore"
opp_stat_list = list(team_year_df_full[team_year_df_full["WTeamID"] == team_id][lose_stat_name]) + list(team_year_df_full[team_year_df_full["LTeamID"] == team_id][win_stat_name])
np.mean(opp_stat_list)

72.85714285714286

In [59]:
win_stat_name = "WScore"
lose_stat_name = "LScore"
stat_list = list(team_year_df_full[team_year_df_full["WTeamID"] == team_id][win_stat_name]) + list(team_year_df_full[team_year_df_full["LTeamID"] == team_id][lose_stat_name])
np.mean(stat_list)

63.07142857142857

In [56]:
team_year_df_full.tail(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,opp_team_id,opp_massey_avg
3011,2003,93,1234,75,1336,55,A,0,22,44,...,13,18,22,10,16,3,5,27,1234,71.327500
3335,2003,100,1326,76,1336,67,A,0,29,58,...,15,12,17,13,10,8,4,20,1326,76.345000
3471,2003,103,1234,84,1336,71,H,0,29,49,...,12,17,13,12,13,9,0,26,1234,71.327500
3651,2003,107,1336,58,1458,57,H,0,21,51,...,11,11,23,12,10,6,2,12,1458,23.972906
3786,2003,110,1278,77,1336,62,H,0,29,64,...,18,18,25,8,25,8,3,14,1278,66.835821
3958,2003,114,1345,79,1336,55,H,0,27,53,...,13,6,18,10,21,1,3,21,1345,27.223587
4112,2003,117,1321,85,1336,79,H,2,27,57,...,21,17,27,11,17,8,2,32,1321,164.210000
4244,2003,121,1276,78,1336,62,A,0,30,50,...,14,13,15,12,10,6,1,10,1276,92.292500
4386,2003,124,1336,74,1231,66,H,0,25,57,...,27,13,22,11,10,4,5,21,1231,28.072115
4523,2003,129,1231,77,1336,49,N,0,26,50,...,7,6,22,9,11,4,2,21,1231,28.072115
